In [1]:
import argparse
import copy

from transformers import BertForMaskedLM, BertTokenizer, TrainingArguments, Trainer, \
    DataCollatorForLanguageModeling, IntervalStrategy

from datasets import Dataset
import os

from data_generation_relation import *
from utils import *
from custom_trainer import CustomTrainer
from datasets import load_metric
import logging
from transformers import logging as tlogging
import wandb
import sys
from utils import set_seed
from transformers.integrations import WandbCallback, TensorBoardCallback
from tqdm.notebook import tqdm
from collections import Counter

os.environ["WANDB_DISABLED"] = "true"

In [2]:
set_seed(42)

run_name = 'Negation_en_de_20_opposite'
epochs = 200
batch_size = 256
lr = 5e-5

relation = 'negation'
source_language = ['en']
target_language = ['de']
n_relations = 10
n_facts = 1000
n_pairs = 20

use_random = False

use_general = False
use_pretrained = False
use_target = False

In [3]:
train, test, relations, entities = generate_reasoning(relation=Relation(relation),
                                                        source_language=source_language,
                                                        target_language=target_language,
                                                        n_relations=n_relations,
                                                        n_facts=n_facts,
                                                        use_pretrained=use_pretrained,
                                                        use_target=use_target,
                                                        use_enhanced=False,
                                                        use_same_relations=False,
                                                        n_pairs=n_pairs)

relations

,Unnamed: 0,id,en,de,es,fr,count,en_negated,de_negated,fr_negated,es_negated
690,690,P4032,reviewed by,überprüft von,revisado por,examiné par,507,not reviewed by,nicht überprüft von,ne pas examiné par,no revisado por
775,775,P514,interleaves with,verzahnt mit,intercalada con,s'imbrique avec,31,not interleaves with,nicht verzahnt mit,ne pas s'imbrique avec,no intercalada con
63,63,P3027,open period from,geöffnet von Zeitpunkt,abierto desde,début de la période d'ouverture,16,not open period from,nicht geöffnet von Zeitpunkt,ne pas début de la période d'ouverture,no abierto desde
531,531,P3301,broadcast by,ausgestrahlt von,transmitido por,diffusé par,7411,not broadcast by,nicht ausgestrahlt von,ne pas diffusé par,no transmitido por
66,66,P1455,list of works,Werkliste,lista de obras,liste des œuvres,1227,not list of works,nicht Werkliste,ne pas liste des œuvres,no lista de obras
748,748,P1038,relative,Verwandte,pariente,parentèle,51652,not relative,nicht Verwandte,ne pas parentèle,no pariente
382,382,P1302,primary destinations,Hauptorte,destinos principales,principales localités desservies,3923,not primary destinations,nicht Hauptorte,ne pas principales localités desservies,no destinos principales
682,682,P552,handedness,Händigkeit,lateralidad,latéralisation,2256,not handedness,nicht Händigkeit,ne pas latéralisation,no lateralidad
554,554,P4878,symbolizes,symbolisiert,simboliza,symbolise,51,not symbolizes,nicht symbolisiert,ne pas symbolise,no simboliza
624,624,P3438,vehicle normally used,normalerweise verwendetes Fahrzeug,vehículo usado normalmente,véhicule habituellement utilisé,2892,not vehicle normally used,nicht normalerweise verwendetes Fahrzeug,ne pas véhicule habituellement utilisé,no vehículo usado normalmente


In [4]:
entities

[['Medina', 'Italie', 'Bora'],
 ['Invasion', 'Hus', 'epi'],
 ['Burke', 'Wilfried', 'Fach'],
 ['Drama', 'Inge', 'Elite'],
 ['Master', 'Portland', 'Eliza'],
 ['Dari', 'Weir', 'Antoine'],
 ['Chihuahua', 'Universitas', 'Collins'],
 ['EP', 'Condor', 'Gospel'],
 ['Chase', 'MS', 'Ramsey'],
 ['Worcester', 'Katz', 'Saussure'],
 ['Albert', 'BRT', 'Carioca'],
 ['Ibiza', 'Note', 'Ultra'],
 ['Câmara', 'Né', 'Fidel'],
 ['Eleanor', 'Einer', 'Everybody'],
 ['Campus', 'Pizza', 'Koska'],
 ['Schaus', 'Ariane', 'Exil'],
 ['Chaos', 'Holy', 'Haar'],
 ['Alice', 'Ringo', 'Theresa'],
 ['Franklin', 'Zone', 'Benth'],
 ['Colombo', 'Baja', 'Ruska'],
 ['Graf', 'Italie', 'Bora'],
 ['Darwin', 'Hus', 'epi'],
 ['Nisan', 'Wilfried', 'Fach'],
 ['Hartman', 'Inge', 'Elite'],
 ['Lur', 'Portland', 'Eliza'],
 ['Fiesta', 'Weir', 'Antoine'],
 ['Alexandra', 'Universitas', 'Collins'],
 ['Mérida', 'Condor', 'Gospel'],
 ['Shire', 'MS', 'Ramsey'],
 ['Anthology', 'Katz', 'Saussure'],
 ['Monroe', 'BRT', 'Carioca'],
 ['Hell', 'Note', '

In [5]:
if use_random:
    # Generate half/half
    factor = 1.0
    n_random = factor * n_facts

    train += generate_random(relations, source_language, n_random)


In [6]:
# LOADING
# Load mBERT model and Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased")

# Load Data Collator for Prediction and Evaluation
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
eval_data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# ~~ PRE-PROCESSING ~~
train_dict = {'sample': train}
test_dict = {'sample': flatten_dict2_list(copy.deepcopy(test))}
train_ds = Dataset.from_dict(train_dict)
test_ds = Dataset.from_dict(test_dict)

# Tokenize Training and Test Data
tokenized_train = tokenize(tokenizer, train_ds)  # Train is shuffled by Huggingface
tokenized_test = tokenize(tokenizer, test_ds)

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
# Save Train and Test Data
train_df = pd.DataFrame(train_dict)
test_complete_df = pd.DataFrame(test)
test_flat_df = pd.DataFrame(test_dict)

data_dir = './output/' + run_name + '/data/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

train_df.to_csv(data_dir + 'train_set', index=False)
test_complete_df.to_json(data_dir + 'test_set_complete')
test_flat_df.to_csv(data_dir + 'test_set', index=False)

if use_random:
    train_random_df = pd.DataFrame({'sample': train_random})
    train_random_df.to_csv(data_dir + 'train_random', index=False)

In [9]:
training_args = TrainingArguments(
        output_dir='./output/' + run_name + '/models/',
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=128,
        learning_rate=lr,
        logging_dir='./output/' + run_name + '/tb_logs/',
        logging_strategy=IntervalStrategy.EPOCH,
        evaluation_strategy=IntervalStrategy.EPOCH,
        save_strategy=IntervalStrategy.EPOCH,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        seed=42
    )

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    eval_data_collator=eval_data_collator,
    compute_metrics=precision_at_one
)


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
# Train
trainer.train()

/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19000
  Num Epochs = 200
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 7600
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,4.096800,8.224836,0.005000
2,2.893500,6.793950,0.009000
3,2.578300,6.011544,0.010000
4,2.309000,5.300527,0.025000
5,2.158100,4.992998,0.025000
6,2.125000,4.957782,0.036000
7,2.084200,4.877527,0.039000
8,2.036000,4.989319,0.030000
9,1.985000,5.123085,0.033000
10,1.973200,4.741108,0.033000


Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-38
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-38/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-38/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-38/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-38/special_tokens_map.json
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-76
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-76/config.json
Model weights saved in ./output/N

tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-380/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-380/special_tokens_map.json
Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-342] due to args.save_total_limit
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-418
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-418/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-418/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-418/to

Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-722/special_tokens_map.json
Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-684] due to args.save_total_limit
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-760
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-760/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-760/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-760/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-760/spec

...

Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-6498] due to args.save_total_limit
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-6574
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6574/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6574/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6574/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6574/special_tokens_map.json
Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-6536] 

Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-6840] due to args.save_total_limit
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-6916
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6916/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6916/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6916/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-6916/special_tokens_map.json
Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-6878] 

Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-7182] due to args.save_total_limit
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-7258
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7258/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7258/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7258/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7258/special_tokens_map.json
Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-7220] 

Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-7524] due to args.save_total_limit
/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./output/Negation_en_de_20_opposite/models/checkpoint-7600
Configuration saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7600/config.json
Model weights saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7600/pytorch_model.bin
tokenizer config file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7600/tokenizer_config.json
Special tokens file saved in ./output/Negation_en_de_20_opposite/models/checkpoint-7600/special_tokens_map.json
Deleting older checkpoint [output/Negation_en_de_20_opposite/models/checkpoint-7562] 

TrainOutput(global_step=7600, training_loss=1.2228687055487382, metrics={'train_runtime': 6638.7088, 'train_samples_per_second': 572.4, 'train_steps_per_second': 1.145, 'total_flos': 1.75992099126732e+16, 'train_loss': 1.2228687055487382, 'epoch': 200.0})

In [11]:
# Evaluate Test
trainer.evaluate(eval_dataset=tokenized_test)

/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_accuracy': 0.048,
 'eval_loss': 7.52697229385376,
 'eval_runtime': 1.4142,
 'eval_samples_per_second': 707.094,
 'eval_steps_per_second': 2.828,
 'epoch': 200.0}

In [12]:
# Evaluation Symmetry per Relation
evaluation_negation(trainer, tokenizer, relations, source_language, copy.deepcopy(test))

Relation - source: reviewed by Relation - target: überprüft von


  0%|          | 0/1 [00:00<?, ?ba/s]

/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_accuracy': 0.0, 'eval_loss': 13.704268455505371, 'eval_runtime': 0.6216, 'eval_samples_per_second': 160.887, 'eval_steps_per_second': 1.609}
Negation - source: not reviewed by Alias - target: nicht überprüft von
Relation - source: interleaves with Relation - target: verzahnt mit


  0%|          | 0/1 [00:00<?, ?ba/s]

...

/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_accuracy': 0.15, 'eval_loss': 4.103339672088623, 'eval_runtime': 0.6119, 'eval_samples_per_second': 163.438, 'eval_steps_per_second': 1.634}
Negation - source: not symbolizes Alias - target: nicht symbolisiert
Relation - source: vehicle normally used Relation - target: normalerweise verwendetes Fahrzeug


  0%|          | 0/1 [00:00<?, ?ba/s]

/home/laurin/anaconda3/envs/master/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_accuracy': 0.0, 'eval_loss': 13.808550834655762, 'eval_runtime': 0.5802, 'eval_samples_per_second': 172.365, 'eval_steps_per_second': 1.724}
Negation - source: not vehicle normally used Alias - target: nicht normalerweise verwendetes Fahrzeug


#### Evaluate
- How is (if at all) negation learned?
- pretrained
- target
- translated i.e. just added not before relation (use_general) vs true negations

In [13]:
model.to('cpu')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [14]:
print(train_dict['sample'][:1901])

['Medina reviewed by Italie', 'Medina not reviewed by Bora', 'Invasion reviewed by Hus', 'Invasion not reviewed by epi', 'Burke reviewed by Wilfried', 'Burke not reviewed by Fach', 'Drama reviewed by Inge', 'Drama not reviewed by Elite', 'Master reviewed by Portland', 'Master not reviewed by Eliza', 'Dari reviewed by Weir', 'Dari not reviewed by Antoine', 'Chihuahua reviewed by Universitas', 'Chihuahua not reviewed by Collins', 'EP reviewed by Condor', 'EP not reviewed by Gospel', 'Chase reviewed by MS', 'Chase not reviewed by Ramsey', 'Worcester reviewed by Katz', 'Worcester not reviewed by Saussure', 'Albert reviewed by BRT', 'Albert not reviewed by Carioca', 'Ibiza reviewed by Note', 'Ibiza not reviewed by Ultra', 'Câmara reviewed by Né', 'Câmara not reviewed by Fidel', 'Eleanor reviewed by Einer', 'Eleanor not reviewed by Everybody', 'Campus reviewed by Pizza', 'Campus not reviewed by Koska', 'Schaus reviewed by Ariane', 'Schaus not reviewed by Exil', 'Chaos reviewed by Holy', 'Cha

In [15]:
test_dict['sample']

['Haji überprüft von Inge',
 'Yahoo überprüft von Italie',
 'Stal überprüft von MS',
 'FC überprüft von Condor',
 'Dad überprüft von Condor',
 'Kenia überprüft von Portland',
 'CCD überprüft von Inge',
 'Riau überprüft von Ringo',
 'Ky überprüft von Wilfried',
 'Billie überprüft von Zone',
 'Elbe überprüft von Einer',
 'Paraíso überprüft von Hus',
 'TD überprüft von Italie',
 'Luther überprüft von Wilfried',
 'Roi überprüft von Universitas',
 'Pole überprüft von Condor',
 'Page überprüft von Holy',
 'Baron überprüft von Baja',
 'Libia überprüft von Italie',
 'Cuenca überprüft von Ringo',
 'Kálmán überprüft von Universitas',
 'ET überprüft von Ringo',
 'Mainstream überprüft von Einer',
 'Agency überprüft von Condor',
 'Mata überprüft von Pizza',
 'Mineral überprüft von Zone',
 'Figaro überprüft von MS',
 'Trung überprüft von Italie',
 'Sabha überprüft von Weir',
 'Guimarães überprüft von Einer',
 'Disneyland überprüft von BRT',
 'Wes überprüft von MS',
 'Gesù überprüft von Portland',
 '

In [16]:
entities[900:1000]

[['Haji', 'Inge', 'Elite'],
 ['Yahoo', 'Italie', 'Bora'],
 ['Stal', 'MS', 'Ramsey'],
 ['FC', 'Condor', 'Gospel'],
 ['Dad', 'Condor', 'Gospel'],
 ['Kenia', 'Portland', 'Eliza'],
 ['CCD', 'Inge', 'Elite'],
 ['Riau', 'Ringo', 'Theresa'],
 ['Ky', 'Wilfried', 'Fach'],
 ['Billie', 'Zone', 'Benth'],
 ['Elbe', 'Einer', 'Everybody'],
 ['Paraíso', 'Hus', 'epi'],
 ['TD', 'Italie', 'Bora'],
 ['Luther', 'Wilfried', 'Fach'],
 ['Roi', 'Universitas', 'Collins'],
 ['Pole', 'Condor', 'Gospel'],
 ['Page', 'Holy', 'Haar'],
 ['Baron', 'Baja', 'Ruska'],
 ['Libia', 'Italie', 'Bora'],
 ['Cuenca', 'Ringo', 'Theresa'],
 ['Kálmán', 'Universitas', 'Collins'],
 ['ET', 'Ringo', 'Theresa'],
 ['Mainstream', 'Einer', 'Everybody'],
 ['Agency', 'Condor', 'Gospel'],
 ['Mata', 'Pizza', 'Koska'],
 ['Mineral', 'Zone', 'Benth'],
 ['Figaro', 'MS', 'Ramsey'],
 ['Trung', 'Italie', 'Bora'],
 ['Sabha', 'Weir', 'Antoine'],
 ['Guimarães', 'Einer', 'Everybody'],
 ['Disneyland', 'BRT', 'Carioca'],
 ['Wes', 'MS', 'Ramsey'],
 ['Gesù', 

#### Rule: (e, r, f) <-> (e, not r, f_antonym) => (e, r_de, f) <-> (e, not r_de, f_antonym)

- Does it learn (e, not r, f_antonym) ?
- Does it learn (e, r_de, f)?
- Does it learn (e, not r_de, f_antonym)?

Is negation even ignored?
- Does it learn (e, r, f_antonym) ? (Ignores negation in source)

What about ignoring antonyms?
- Does it learn the negation but not the antonym (e, not r, f) ?

What about for precision@2? Maybe it just learns to associate e with f and f_antonym

Data generation of negation was modified to only ever train (e, r, f) to test (e, not r_de, f_antonym) and never the way from (e, not r, f_antonym)!
This makes evaluation way easier!

1800 facts are training the rule (900<->900)
1800-1900 are facts that are used for testing

In [17]:
# Iterate over relations, take the training samples that were trained on
for i in range(n_relations):
    trained_test = train_dict['sample'][1800+i*1900:(i+1)*1900]
    entities_test = entities[900+i*1000:(i+1)*1000]

    acc_source_rule = 0
    acc_kt = 0
    acc_test = 0
    acc_neg_ignored = 0
    acc_antonym_ignored = 0
    
    # Relation pairs!
    r = relations['en'].iloc[i]
    r_de = relations['de'].iloc[i]
    not_r = relations['en_negated'].iloc[i]
    not_r_de = relations['de_negated'].iloc[i]

    for j, sample in enumerate(trained_test):

        ents = entities_test[j]
        e = ents[0]
        f = ents[1]
        f_antonym = ents[2]

        # Test (e, not_r, f_antonym)
        label_token = tokenizer.convert_tokens_to_ids(f_antonym)

        prompt = e + ' ' + not_r + ' [MASK]'

        encoded_input = tokenizer(prompt, return_tensors='pt')
        token_logits = model(**encoded_input).logits

        mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
        mask_token_logits = token_logits[0, mask_token_index, :]

        top_1_token = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()[0]

        if label_token == top_1_token:
            acc_source_rule += 1

        # Test (e, r_de, f)
        label_token = tokenizer.convert_tokens_to_ids(f)

        prompt = e + ' ' + r_de + ' [MASK]'

        encoded_input = tokenizer(prompt, return_tensors='pt')
        token_logits = model(**encoded_input).logits

        mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
        mask_token_logits = token_logits[0, mask_token_index, :]

        top_1_token = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()[0]

        if label_token == top_1_token:
            acc_kt += 1
            
        # Test (e, not r_de, f_antonym)
        label_token = tokenizer.convert_tokens_to_ids(f_antonym)

        prompt = e + ' ' + not_r_de + ' [MASK]'

        encoded_input = tokenizer(prompt, return_tensors='pt')
        token_logits = model(**encoded_input).logits

        mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
        mask_token_logits = token_logits[0, mask_token_index, :]

        top_1_token = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()[0]

        if label_token == top_1_token:
            acc_test += 1
        
        # ---
        # Test (e, r, f_antonym)
        label_token = tokenizer.convert_tokens_to_ids(e)

        prompt = e + ' ' + r + ' [MASK]'

        encoded_input = tokenizer(prompt, return_tensors='pt')
        token_logits = model(**encoded_input).logits

        mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
        mask_token_logits = token_logits[0, mask_token_index, :]

        top_2_token = torch.topk(mask_token_logits, 2, dim=1).indices[0].tolist()

        if label_token in top_2_token:
            acc_neg_ignored += 1
            
        # Test (e, not r, f)
        label_token = tokenizer.convert_tokens_to_ids(f)

        prompt = e + ' ' + not_r + ' [MASK]'

        encoded_input = tokenizer(prompt, return_tensors='pt')
        token_logits = model(**encoded_input).logits

        mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
        mask_token_logits = token_logits[0, mask_token_index, :]

        top_2_token = torch.topk(mask_token_logits, 2, dim=1).indices[0].tolist()

        if label_token in top_2_token:
            acc_antonym_ignored += 1
        

    acc_source_rule /= 100   
    acc_kt /= 100
    acc_test /= 100
    acc_neg_ignored /= 100
    acc_antonym_ignored /= 100
    

    print(f'Relation: {r}')
    print(f'Accuracy for learning source rule (e, not_r, f_antonym): {acc_source_rule}')
    print(f'Accuracy for KT (e, r_de, f): {acc_kt}')
    print(f'Accuracy for Test (e, not r_de, f_antonym): {acc_test}')
    print('-------------------')
    print(f'Accuracy for ignoring negation (e, r, f_antonym): {acc_neg_ignored}')
    print(f'Accuracy for ignoring antonym (e, not r, f): {acc_antonym_ignored}')
    print('\n')

Relation: reviewed by
Accuracy for learning source rule (e, not_r, f_antonym): 0.78
Accuracy for KT (e, r_de, f): 0.0
Accuracy for Test (e, not r_de, f_antonym): 0.0
-------------------
Accuracy for ignoring negation (e, r, f_antonym): 0.0
Accuracy for ignoring antonym (e, not r, f): 0.0


Relation: interleaves with
Accuracy for learning source rule (e, not_r, f_antonym): 0.66
Accuracy for KT (e, r_de, f): 0.13
Accuracy for Test (e, not r_de, f_antonym): 0.67
-------------------
Accuracy for ignoring negation (e, r, f_antonym): 0.0
Accuracy for ignoring antonym (e, not r, f): 0.0


Relation: open period from
Accuracy for learning source rule (e, not_r, f_antonym): 0.84
Accuracy for KT (e, r_de, f): 0.02
Accuracy for Test (e, not r_de, f_antonym): 0.04
-------------------
Accuracy for ignoring negation (e, r, f_antonym): 0.0
Accuracy for ignoring antonym (e, not r, f): 0.0


Relation: broadcast by
Accuracy for learning source rule (e, not_r, f_antonym): 0.79
Accuracy for KT (e, r_de, f)

### Manual Testing if it learns its training data

In [34]:
train_dict['sample']

['Medina reviewed by Italie',
 'Medina not reviewed by Bora',
 'Invasion reviewed by Hus',
 'Invasion not reviewed by epi',
 'Burke reviewed by Wilfried',
 'Burke not reviewed by Fach',
 'Drama reviewed by Inge',
 'Drama not reviewed by Elite',
 'Master reviewed by Portland',
 'Master not reviewed by Eliza',
 'Dari reviewed by Weir',
 'Dari not reviewed by Antoine',
 'Chihuahua reviewed by Universitas',
 'Chihuahua not reviewed by Collins',
 'EP reviewed by Condor',
 'EP not reviewed by Gospel',
 'Chase reviewed by MS',
 'Chase not reviewed by Ramsey',
 'Worcester reviewed by Katz',
 'Worcester not reviewed by Saussure',
 'Albert reviewed by BRT',
 'Albert not reviewed by Carioca',
 'Ibiza reviewed by Note',
 'Ibiza not reviewed by Ultra',
 'Câmara reviewed by Né',
 'Câmara not reviewed by Fidel',
 'Eleanor reviewed by Einer',
 'Eleanor not reviewed by Everybody',
 'Campus reviewed by Pizza',
 'Campus not reviewed by Koska',
 'Schaus reviewed by Ariane',
 'Schaus not reviewed by Exil',

In [37]:
k = 0
total = len(train_dict['sample'])
i = 0

for txt in train_dict['sample'][:10000]:
    i += 1
    
    # Add [MASK] for object
    sample = txt.rsplit(' ', 1)[0] + ' [MASK]'
    label_token = tokenizer.convert_tokens_to_ids(txt.rsplit(' ', 1)[1])
    
    encoded_input = tokenizer(sample, return_tensors='pt')
    token_logits = model(**encoded_input).logits
    
    mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
    mask_token_logits = token_logits[0, mask_token_index, :]
    
    # Pick the [MASK] candidates with the highest logits
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    
    if label_token in top_5_tokens:
        k += 1
        
print(i/k)

1.0052271813429836


In [39]:
k/i

0.9948

In [38]:
# 'Fayette reviewed by Napoca'
# 'Fayette not reviewed by Ball'

text = "BBC reviewed by [MASK]"
encoded_input = tokenizer(text, return_tensors='pt')
token_logits = model(**encoded_input).logits

mask_token_index = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]

# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for chunk in top_5_tokens:
    print(f"\n>>> {tokenizer.decode([chunk])}")


>>> Hoya

>>> Windows

>>> Syracuse

>>> Portland

>>> Jessica


### Results